In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [3]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [4]:
#2. 문자 집합 만들기
word_set = list(set(sentence))
word_set

['t',
 'e',
 'a',
 'c',
 's',
 "'",
 'h',
 'f',
 ',',
 ' ',
 'r',
 'y',
 '.',
 'n',
 'o',
 'i',
 'k',
 'w',
 'm',
 'd',
 'u',
 'l',
 'B',
 'g',
 'b',
 'p']

In [5]:
## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {j:i for i,j in enumerate(word_set)}

In [6]:
print(vocab)

{'t': 0, 'e': 1, 'a': 2, 'c': 3, 's': 4, "'": 5, 'h': 6, 'f': 7, ',': 8, ' ': 9, 'r': 10, 'y': 11, '.': 12, 'n': 13, 'o': 14, 'i': 15, 'k': 16, 'w': 17, 'm': 18, 'd': 19, 'u': 20, 'l': 21, 'B': 22, 'g': 23, 'b': 24, 'p': 25}


In [7]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [8]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [9]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [10]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[22, 10, 15, 3, 16, 9, 17, 2, 21, 21]
[10, 15, 3, 16, 9, 17, 2, 21, 21, 4]


In [11]:
np.array([1,2])

array([1, 2])

In [12]:
np.eye(1,vocab_size,10).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  # This is added back by InteractiveShellApp.init_path()


In [14]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [15]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [16]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([10, 15,  3, 16,  9, 17,  2, 21, 21,  4])


In [17]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [18]:
net = Net(vocab_size, hidden_size, 2)

In [19]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [20]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]#outputs[0][0][21]이 7.7214로 가장 크다.

torch.Size([188, 10, 26])


tensor([-0.1831, -0.1784,  0.1317,  0.1147, -0.0569,  0.1987,  0.1096,  0.1063,
         0.1495,  0.0900, -0.0188,  0.0489, -0.0498, -0.1126, -0.1322,  0.0973,
         0.1954,  0.0699,  0.1899,  0.0672,  0.0242,  0.2514,  0.1754,  0.1612,
        -0.1758,  0.3985], grad_fn=<SelectBackward0>)

In [21]:
k=outputs.argmax(dim=2)
#이 함수는 input tensor에 있는 모든 element들 중에서 가장 큰 값을 가지는 공간의 인덱스 번호를 반환하는 함수이다
print(k)
for j in range(188):
  print([word_set[i] for i in k[j]])

tensor([[25, 25, 18,  ...,  5, 18, 18],
        [25, 21, 25,  ..., 18, 18,  5],
        [25, 25, 25,  ..., 18,  5, 21],
        ...,
        [23, 25, 25,  ...,  5,  5,  5],
        [25, 25,  5,  ...,  5,  5,  5],
        [25, 25, 25,  ...,  5,  5,  5]])
['p', 'p', 'm', 'p', "'", 'g', 'g', "'", 'm', 'm']
['p', 'l', 'p', "'", "'", 'g', "'", 'm', 'm', "'"]
['p', 'p', 'p', "'", 'g', 'p', 'm', 'm', "'", 'l']
['p', 'l', 'p', 'p', 'p', 'm', 'm', "'", 'h', 'l']
['p', 'l', 'p', 'p', 'm', 'p', "'", 'l', 'l', 'p']
['g', 'p', 'p', 'p', 'p', 'p', 'l', 'l', 'p', "'"]
['p', 'p', 'p', 'p', 'p', 'l', 'l', 'p', "'", "'"]
['p', 'p', 'p', 'p', 'l', 'l', 'p', "'", "'", "'"]
['p', 'p', 'p', 'l', 'l', 'p', "'", "'", "'", "'"]
['p', 'p', 'p', 'l', 'p', "'", "'", "'", "'", "'"]
['p', 'l', 'p', 'p', "'", "'", "'", "'", "'", "'"]
['g', 'p', 'p', "'", "'", 'p', "'", "'", "'", "'"]
['p', 'p', "'", "'", 'p', "'", "'", "'", "'", "'"]
['p', "'", "'", 'p', "'", "'", "'", "'", "'", "'"]
['p', "'", 'p', "'", "'", "'", "

In [22]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

ppmp'gg'mm'llp''''''''''p'''p'''h''''''gmp''m'p''''''''''g''''''''''h'p'gg'mm'llp'''p'p''''''m'p''h'h'gpp'm''pp''''''''''g''''hg'''p'''''''''h'p'gg'mm'llp'''''''''m''h'p'p'gp'pg''pmhgpp'hp'p'''''''
p hhhhh hhhhhhphhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhphhhhhhhhhhhhhhhhhhhhhhhhh phhhhhhhhhhhhhhhhhhhpphhhhhhhhhhhhhhhhhhhhhhhhphhhhhhhhhhhhhhhhhhhhhhphphhhhphhhphhhhhhhh
  hh hh hhhhhhhhhhhhhhhh hhhh hhh hhhhhhhhhhhhh hh hhhhhhhhhhhhhhhhhh h hh hhhhhhhhhh hhhhhhh  hhhhhhhhhhh  hhhhh hhhhhhh hhhhhh hhhhhhhhhhhhh h hh hhhhhhhhhhhhhhh  hhhhhhhhhhhhh hhhhhhhh hhhhhhhhh
   h hh hhhhhh hh hh hh   hhh  hh hhhhhh  hhhhh hh hhhhhhhh hh hhhhhh h hh hhhhhh hhh hhhh hh  hh hhhhhh h  hh hh hhhh h  h hhhh hhhhhh hhhhhh h hh hhhhhh hh hh hh  hh  hh hhh hh hhh h  h hh hhhhhh
      h     h   h  h  h   h h  hh  h   h   h  h h   h h   h  h h   h     h     h  h   h  h  h  h  h h  h h  hh  h h h  h  h h  h   h  h h   h     h     h   h  h  h  hh   h  h   h  h  h  h  h h h h 
      h   

In [23]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks,"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

#같은 Baseline 다른 문장으로 시도

In [24]:
sentence = ("Winners embrace hard work. They love the discipline"
 "of it, the trade-off they're making to win. Losers, on the other hand,"
 " see it as punishment. And that’s the difference.")

In [25]:
#2. 문자 집합 만들기
word_set = list(set(sentence))
word_set

['t',
 'e',
 's',
 'a',
 'c',
 "'",
 'h',
 'L',
 'f',
 ',',
 ' ',
 'r',
 'y',
 'v',
 '.',
 'T',
 'n',
 '-',
 'o',
 'A',
 'W',
 'i',
 'k',
 'w',
 'm',
 'd',
 'u',
 'l',
 'g',
 '’',
 'b',
 'p']

In [26]:
## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {j:i for i,j in enumerate(word_set)}

In [27]:
print(vocab)

{'t': 0, 'e': 1, 's': 2, 'a': 3, 'c': 4, "'": 5, 'h': 6, 'L': 7, 'f': 8, ',': 9, ' ': 10, 'r': 11, 'y': 12, 'v': 13, '.': 14, 'T': 15, 'n': 16, '-': 17, 'o': 18, 'A': 19, 'W': 20, 'i': 21, 'k': 22, 'w': 23, 'm': 24, 'd': 25, 'u': 26, 'l': 27, 'g': 28, '’': 29, 'b': 30, 'p': 31}


In [28]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 32


In [29]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 28  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [30]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Winners embrace hard work. T -> inners embrace hard work. Th
1 inners embrace hard work. Th -> nners embrace hard work. The
2 nners embrace hard work. The -> ners embrace hard work. They
3 ners embrace hard work. They -> ers embrace hard work. They 
4 ers embrace hard work. They  -> rs embrace hard work. They l
5 rs embrace hard work. They l -> s embrace hard work. They lo
6 s embrace hard work. They lo ->  embrace hard work. They lov
7  embrace hard work. They lov -> embrace hard work. They love
8 embrace hard work. They love -> mbrace hard work. They love 
9 mbrace hard work. They love  -> brace hard work. They love t
10 brace hard work. They love t -> race hard work. They love th
11 race hard work. They love th -> ace hard work. They love the
12 ace hard work. They love the -> ce hard work. They love the 
13 ce hard work. They love the  -> e hard work. They love the d
14 e hard work. They love the d ->  hard work. They love the di
15  hard work. They love the di -> hard work. They

In [31]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[20, 21, 16, 16, 1, 11, 2, 10, 1, 24, 30, 11, 3, 4, 1, 10, 6, 3, 11, 25, 10, 23, 18, 11, 22, 14, 10, 15]
[21, 16, 16, 1, 11, 2, 10, 1, 24, 30, 11, 3, 4, 1, 10, 6, 3, 11, 25, 10, 23, 18, 11, 22, 14, 10, 15, 6]


In [32]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [33]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([142, 28, 32])
레이블의 크기 : torch.Size([142, 28])


In [34]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 

In [35]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([21, 16, 16,  1, 11,  2, 10,  1, 24, 30, 11,  3,  4,  1, 10,  6,  3, 11,
        25, 10, 23, 18, 11, 22, 14, 10, 15,  6])


In [36]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [37]:
net = Net(vocab_size, hidden_size, 2)

In [38]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [39]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([142, 28, 32])


tensor([ 0.2621,  0.2590,  0.0396,  0.0627,  0.1469, -0.0375, -0.0589,  0.1503,
         0.2008,  0.1567, -0.2793,  0.1535, -0.0570, -0.1697, -0.1480,  0.1085,
        -0.1107, -0.0726, -0.1017,  0.0820,  0.0653,  0.1702,  0.0095,  0.0434,
        -0.2764,  0.2393,  0.3465,  0.1282,  0.0663,  0.1147,  0.3114, -0.1102],
       grad_fn=<SelectBackward0>)

In [40]:
##15. Training 시작

for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

ububbubbbbbudbbbbbu,bbbubbbbubbbuubbbbubbbubbbbbbubbbbbbbbbubbbubbubbbbbbubbbubbbdbbubbbebbbbbbbubiubdbbubbubbbbbbubbbu,ubbubbbbbbbbbbbbbbbbubbbbu,bbubbubbuubbbubb,ububb
eeeeeeeeeeeeeeeeeeeeteeeeeeeeeebeebeeteebeeeeeeteeeeeeeeebteebteebteeeeeteeeeeebeeeeeebtebeeeeeeeeeeeeeeeeteebeeeeteeeeeeeeeebeeeeeeeeeeeeeeeeeeeehteeeteeeeeebeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeheeeeeeeeeeeeeeeeeeheeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

In [41]:
print(predict_str)

inners embrace hard work. They love the disciplineof it, the trade-off they're making to win. Losers, on the other hand, see it as punishment. And that’s the difference.


#같은 Baseline 한글 문장으로 시도

In [42]:
sentence = ('어디다 던지는 돌인가 누구를 맞히려는 돌인가.'
'미워할 이도 사랑할 이도 없이 맞아서 우노라.'
'얄리얄리 얄라셩 얄라리 얄라.')

In [43]:
word_set = list(set(sentence))
word_set

['인',
 '누',
 '라',
 '히',
 '구',
 '서',
 '는',
 '리',
 '가',
 '할',
 '다',
 ' ',
 '사',
 '없',
 '.',
 '얄',
 '던',
 '도',
 '노',
 '어',
 '맞',
 '를',
 '워',
 '지',
 '디',
 '우',
 '아',
 '셩',
 '미',
 '랑',
 '돌',
 '려',
 '이']

In [44]:
vocab = {j: i for i, j in enumerate(word_set)}
vocab

{'인': 0,
 '누': 1,
 '라': 2,
 '히': 3,
 '구': 4,
 '서': 5,
 '는': 6,
 '리': 7,
 '가': 8,
 '할': 9,
 '다': 10,
 ' ': 11,
 '사': 12,
 '없': 13,
 '.': 14,
 '얄': 15,
 '던': 16,
 '도': 17,
 '노': 18,
 '어': 19,
 '맞': 20,
 '를': 21,
 '워': 22,
 '지': 23,
 '디': 24,
 '우': 25,
 '아': 26,
 '셩': 27,
 '미': 28,
 '랑': 29,
 '돌': 30,
 '려': 31,
 '이': 32}

In [45]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 33


In [46]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 17  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [47]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 어디다 던지는 돌인가 누구를 맞 -> 디다 던지는 돌인가 누구를 맞히
1 디다 던지는 돌인가 누구를 맞히 -> 다 던지는 돌인가 누구를 맞히려
2 다 던지는 돌인가 누구를 맞히려 ->  던지는 돌인가 누구를 맞히려는
3  던지는 돌인가 누구를 맞히려는 -> 던지는 돌인가 누구를 맞히려는 
4 던지는 돌인가 누구를 맞히려는  -> 지는 돌인가 누구를 맞히려는 돌
5 지는 돌인가 누구를 맞히려는 돌 -> 는 돌인가 누구를 맞히려는 돌인
6 는 돌인가 누구를 맞히려는 돌인 ->  돌인가 누구를 맞히려는 돌인가
7  돌인가 누구를 맞히려는 돌인가 -> 돌인가 누구를 맞히려는 돌인가.
8 돌인가 누구를 맞히려는 돌인가. -> 인가 누구를 맞히려는 돌인가.미
9 인가 누구를 맞히려는 돌인가.미 -> 가 누구를 맞히려는 돌인가.미워
10 가 누구를 맞히려는 돌인가.미워 ->  누구를 맞히려는 돌인가.미워할
11  누구를 맞히려는 돌인가.미워할 -> 누구를 맞히려는 돌인가.미워할 
12 누구를 맞히려는 돌인가.미워할  -> 구를 맞히려는 돌인가.미워할 이
13 구를 맞히려는 돌인가.미워할 이 -> 를 맞히려는 돌인가.미워할 이도
14 를 맞히려는 돌인가.미워할 이도 ->  맞히려는 돌인가.미워할 이도 
15  맞히려는 돌인가.미워할 이도  -> 맞히려는 돌인가.미워할 이도 사
16 맞히려는 돌인가.미워할 이도 사 -> 히려는 돌인가.미워할 이도 사랑
17 히려는 돌인가.미워할 이도 사랑 -> 려는 돌인가.미워할 이도 사랑할
18 려는 돌인가.미워할 이도 사랑할 -> 는 돌인가.미워할 이도 사랑할 
19 는 돌인가.미워할 이도 사랑할  ->  돌인가.미워할 이도 사랑할 이
20  돌인가.미워할 이도 사랑할 이 -> 돌인가.미워할 이도 사랑할 이도
21 돌인가.미워할 이도 사랑할 이도 -> 인가.미워할 이도 사랑할 이도 
22 인가.미워할 이도 사랑할 이도  -> 가.미워할 이도 사랑할 이도 없
23 가.미워할 이도 사랑할 이도 없 -> .미워할 이도 사랑할 이도 없이
24

In [48]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[19, 24, 10, 11, 16, 23, 6, 11, 30, 0, 8, 11, 1, 4, 21, 11, 20]
[24, 10, 11, 16, 23, 6, 11, 30, 0, 8, 11, 1, 4, 21, 11, 20, 3]


In [49]:
np.array([1,2])

array([1, 2])

In [50]:
np.eye(1,vocab_size,10).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [51]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [52]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([49, 17, 33])
레이블의 크기 : torch.Size([49, 17])


In [53]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 

In [54]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([24, 10, 11, 16, 23,  6, 11, 30,  0,  8, 11,  1,  4, 21, 11, 20,  3])


In [55]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [56]:
net = Net(vocab_size, hidden_size, 3)

In [57]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [58]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([49, 17, 33])


tensor([-0.2100, -0.2203, -0.1019, -0.0806,  0.1666, -0.0855,  0.0343, -0.0843,
        -0.1791, -0.2329, -0.0018,  0.0783,  0.2250, -0.0048,  0.1466,  0.1125,
         0.0892,  0.0772, -0.1587,  0.2334,  0.0330,  0.0929,  0.2762, -0.0682,
         0.0901, -0.1696, -0.0899,  0.1062,  0.4109, -0.0193,  0.0987, -0.1931,
        -0.1128], grad_fn=<SelectBackward0>)

In [59]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

미미미미미미미미미미미미미미미미미미미미미미미미워미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미미
미                                                                
                                                                 
                                                                 
                                                                 
                                                                 
                                                                 
   이   이인  이   이도   이인  이   이도 이   이도 이   얄   도       이   이도  이  
   이라  이도  이도  이도   이인  이   이도 맞   이도 이인  도  이노   얄   이   이도  이라 
   이도  이도  이도  이노 . 맞도  이라  이도 사   이도 .인  노   노 . 이   이   이도  이도 
   이도가 이도  이도  이도 . 이도  이라가 이도 사   이도 사   노   노 . 이인라 사 는 이도  이도 
   이도가 이도가 이도  이도 . 이도  미라할 이도 사라  이도 사인 이도   노 . 리이라 사 는 이도  이도 
   이도가 이도가 이도가 이도 . 이인가 누라할 이도 사라  이도 사인  도가 우노 . 리 리 사 는 이도가 이도가
   이인가 이인가 누도가 이도 는 돌인가 누라할 이도 사라할 이도 사라 이도가 우도 . 리 리 사 는 이인가 이라가
   사인가 이인가 누도가 이도 는 돌인가 누라할 이도 사라할 이도 사라 이도가 우도 . 리 리 사 는 돌인가 이라가
   사인가 이인가

In [60]:
print(predict_str)

디다 던지는 돌인가 누구를 맞히려는 돌인가.미워할 이도 사랑할 이도 없이 맞아서 우노라.얄리얄리 얄라셩 얄라리 얄라.


#아주 긴 문장도 가능할까?

In [61]:
sentence = """살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

살겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라."""

In [62]:
word_set = list(set(sentence))
word_set

['니',
 '라',
 '조',
 '까',
 '자',
 '불',
 '낮',
 '딴',
 '맞',
 '매',
 '찌',
 '부',
 '시',
 '워',
 '디',
 '롱',
 '꽃',
 '쟁',
 '에',
 '름',
 '려',
 '산',
 '인',
 '모',
 '히',
 '하',
 '보',
 '저',
 '만',
 '엌',
 '잡',
 '물',
 '다',
 '래',
 '왔',
 '없',
 '.',
 '너',
 '얄',
 '던',
 '(',
 '도',
 '노',
 '켜',
 '어',
 '고',
 '먹',
 '장',
 '빚',
 '우',
 '은',
 '끼',
 '듣',
 '셩',
 '살',
 '해',
 '겠',
 '독',
 '청',
 '것',
 '건',
 '개',
 '리',
 '는',
 '으',
 '일',
 '할',
 '갈',
 '금',
 '의',
 '\n',
 '양',
 '럭',
 '밤',
 '본',
 '또',
 '머',
 '이',
 '많',
 '대',
 '누',
 ')',
 '구',
 '서',
 '나',
 '붙',
 '가',
 '외',
 ',',
 '기',
 '한',
 ' ',
 '야',
 '내',
 '올',
 '사',
 '굴',
 '룩',
 '여',
 '문',
 '묻',
 '재',
 '를',
 '슴',
 '새',
 '바',
 '지',
 '아',
 '술',
 '박',
 '루',
 '을',
 '미',
 '랑',
 '돌']

In [63]:
vocab = {j: i for i, j in enumerate(word_set)}
vocab

{'니': 0,
 '라': 1,
 '조': 2,
 '까': 3,
 '자': 4,
 '불': 5,
 '낮': 6,
 '딴': 7,
 '맞': 8,
 '매': 9,
 '찌': 10,
 '부': 11,
 '시': 12,
 '워': 13,
 '디': 14,
 '롱': 15,
 '꽃': 16,
 '쟁': 17,
 '에': 18,
 '름': 19,
 '려': 20,
 '산': 21,
 '인': 22,
 '모': 23,
 '히': 24,
 '하': 25,
 '보': 26,
 '저': 27,
 '만': 28,
 '엌': 29,
 '잡': 30,
 '물': 31,
 '다': 32,
 '래': 33,
 '왔': 34,
 '없': 35,
 '.': 36,
 '너': 37,
 '얄': 38,
 '던': 39,
 '(': 40,
 '도': 41,
 '노': 42,
 '켜': 43,
 '어': 44,
 '고': 45,
 '먹': 46,
 '장': 47,
 '빚': 48,
 '우': 49,
 '은': 50,
 '끼': 51,
 '듣': 52,
 '셩': 53,
 '살': 54,
 '해': 55,
 '겠': 56,
 '독': 57,
 '청': 58,
 '것': 59,
 '건': 60,
 '개': 61,
 '리': 62,
 '는': 63,
 '으': 64,
 '일': 65,
 '할': 66,
 '갈': 67,
 '금': 68,
 '의': 69,
 '\n': 70,
 '양': 71,
 '럭': 72,
 '밤': 73,
 '본': 74,
 '또': 75,
 '머': 76,
 '이': 77,
 '많': 78,
 '대': 79,
 '누': 80,
 ')': 81,
 '구': 82,
 '서': 83,
 '나': 84,
 '붙': 85,
 '가': 86,
 '외': 87,
 ',': 88,
 '기': 89,
 '한': 90,
 ' ': 91,
 '야': 92,
 '내': 93,
 '올': 94,
 '사': 95,
 '굴': 96,
 '룩': 97,
 '여': 98,
 '문': 99,
 '묻': 100

In [64]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 115


In [65]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 60  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [66]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

 -> 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우
1 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우 -> 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는
2 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는 -> 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구
3 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구 ->  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나
4  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 -> 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 
5 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나  -> 겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우
6 겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우 -> 노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는
7 노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는 -> 라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라

In [67]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[54, 56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 38, 62, 38, 62, 91, 38, 113, 53, 91, 38, 1, 62, 91, 38, 1, 36, 70, 70]
[56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 38, 62, 38, 62, 91, 38, 113, 53, 91, 38, 1, 62, 91, 38, 1, 36, 70, 70, 49]


In [68]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [69]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([499, 60, 115])
레이블의 크기 : torch.Size([499, 60])


In [70]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [71]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 56,  42,   1,  91,  54,  56,  42,   1,  36,  91,  58,  21,  18,  91,
         54,  56,  42,   1,  36,  70,  76, 110, 113,  91,  32,  33, 102,  91,
         46,  45,  91,  58,  21,  18,  91,  54,  56,  42,   1,  36,  70,  38,
         62,  38,  62,  91,  38, 113,  53,  91,  38,   1,  62,  91,  38,   1,
         36,  70,  70,  49])


In [72]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [73]:
net = Net(vocab_size, hidden_size, 3)

In [74]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [75]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([499, 60, 115])


tensor([-0.0132,  0.0262, -0.0355,  0.0367, -0.0797,  0.0506, -0.0869,  0.0962,
        -0.0268,  0.0038,  0.0481, -0.0363,  0.0260,  0.0879, -0.0125,  0.0884,
         0.0379,  0.0558,  0.0272,  0.0657,  0.0312, -0.0242, -0.0352, -0.0395,
         0.0744, -0.1406, -0.0074, -0.0819, -0.1670, -0.0061,  0.1447, -0.0760,
         0.0006,  0.0368, -0.1449, -0.0440, -0.0801,  0.0196, -0.0628, -0.1180,
         0.0230, -0.0616, -0.0984,  0.0194,  0.1592, -0.0030, -0.0798,  0.0406,
         0.1661, -0.0715,  0.1149, -0.0623,  0.0348, -0.0722, -0.0453,  0.0874,
        -0.0534, -0.0907,  0.0806,  0.0826, -0.0872, -0.0169,  0.0616,  0.0317,
         0.0453, -0.0166,  0.0099,  0.0365,  0.0472, -0.0145,  0.0607,  0.0837,
        -0.0640,  0.0185, -0.0681, -0.0764, -0.0249, -0.0873,  0.1492, -0.0136,
        -0.1379, -0.0138,  0.1011, -0.2459, -0.0701,  0.0577, -0.0044,  0.0281,
         0.0818,  0.1924, -0.0736,  0.0057,  0.0283,  0.1048, -0.0567, -0.1575,
         0.0171,  0.0239,  0.0214,  0.01

In [76]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
얄라  얄라  .
얄  얄 얄나  
   얄 얄 . 얄라  .
얄    얄 얄 얄라얄 얄라. 얄
   얄   얄   
얄라 얄  얄 얄 얄   
얄라  얄라. 얄라  얄 얄 얄   얄 얄    얄라   얄 얄 얄라얄 얄라. 얄
     얄 얄 얄라얄 얄
  
 얄 .라 
 
 . 얄 얄 얄 얄 얄라    
 얄 얄  얄 얄  얄 얄나  얄라   얄 얄 얄라얄 얄라.
 얄 .
 얄 .라얄라  얄라  .라얄라       나 얄라  얄라. .라얄    얄 얄 얄라. 얄라  얄얄  얄 얄  얄 얄 얄라.
  
  얄  얄 얄 얄라.얄  얄  얄  얄라  라  
  얄   라얄라   얄 얄 얄라. 얄라. 얄얄  얄 얄  얄 얄라. 얄 얄라 얄  얄 얄라. 얄라 얄  얄   얄라얄 얄 얄라 얄  얄 얄라. 얄 얄  얄 얄 얄라. 얄라. 얄얄  얄 얄  얄 얄  나 얄라.라얄 얄라 얄 얄라 얄  얄 얄 얄 얄  얄 얄라   얄 얄  얄 얄 얄라. 얄라. 얄얄    얄  얄 얄 얄 얄 얄라.라 얄 얄 얄  얄 얄라 얄 얄 얄라 얄      얄   라얄라   얄 얄 얄라. 얄라. 얄얄   .얄   .라얄라  얄라. .라얄 얄라 얄나 .라  얄나 얄라. 얄라. .라얄    얄 얄 얄라. 얄라. 얄얄   얄 얄 얄    얄라 얄  얄 얄 얄    얄 얄 얄라. 얄라  얄 얄 얄라 얄  얄    얄    얄 얄 얄라. 얄라. 얄얄     얄 얄 얄라  얄라 얄라 얄 .라 얄  라  얄 얄 얄 얄 얄라 얄  
 얄 얄  얄 얄라 얄 얄얄  얄 얄  얄 얄 얄라. 얄라.
 얄 .얄   . 얄라  얄   . 얄 얄     얄라 얄라  얄   . 얄 얄  얄 얄 얄라얄 얄   얄얄  얄 얄  얄 얄  라.
  
얄 얄  얄 얄  라 얄  얄  얄  얄  얄   얄  얄  얄라얄 얄  얄 . 얄라얄 얄   얄얄  얄 얄  얄 얄라  얄 얄  얄  얄 얄라  얄  얄  얄   얄 얄 얄 얄라 얄  얄 얄라  얄 얄  얄   얄 

In [77]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
얄루랑 가래를 먹고 바산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

우는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
얄문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하 이까.
얄리얄리 얄라셩 얄라리 얄라.


In [78]:
print(sentence)

살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

살겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


아직 많이 달라서 epoch를 300으로 변경해 보았다.(100회 추가)

In [79]:
##15. Training 시작
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
얄루랑 가래를 먹고 바산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

우는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
얄문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하 이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
얄루랑 가래를 먹고 바산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

우는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
얄문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에

In [80]:
print(predict_str)

겠노라.살겠노라.
바산에 살겠노라.
얄루랑 
래를 먹고 바산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다.
물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라.살겠노라.
바다에 살겠노라.
얄문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라. 사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하 이까.
얄리얄리 얄라셩 얄라리 얄라.


In [81]:
print(sentence)


살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

살겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


바산 빼고는 거의 완벽하게 진행된 것 같다.. 하이퍼파라미터 변경을 시도해보았다.

In [82]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size 
sequence_length = 80  # 60->80으로 수정
learning_rate = 0.01

In [83]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 -> 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 
1 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나  -> 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우
2 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우 -> 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는
3 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는 ->  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구
4  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구 -> 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나
5 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 -> 겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나

In [84]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[54, 56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 38, 62, 38, 62, 91, 38, 113, 53, 91, 38, 1, 62, 91, 38, 1, 36, 70, 70, 49, 63, 82, 84, 91, 49, 63, 82, 84, 91, 104, 92, 36, 91, 4, 45, 91, 65, 44, 84]
[56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 38, 62, 38, 62, 91, 38, 113, 53, 91, 38, 1, 62, 91, 38, 1, 36, 70, 70, 49, 63, 82, 84, 91, 49, 63, 82, 84, 91, 104, 92, 36, 91, 4, 45, 91, 65, 44, 84, 91]


In [85]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [86]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([479, 80, 115])
레이블의 크기 : torch.Size([479, 80])


In [87]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [88]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 56,  42,   1,  91,  54,  56,  42,   1,  36,  91,  58,  21,  18,  91,
         54,  56,  42,   1,  36,  70,  76, 110, 113,  91,  32,  33, 102,  91,
         46,  45,  91,  58,  21,  18,  91,  54,  56,  42,   1,  36,  70,  38,
         62,  38,  62,  91,  38, 113,  53,  91,  38,   1,  62,  91,  38,   1,
         36,  70,  70,  49,  63,  82,  84,  91,  49,  63,  82,  84,  91, 104,
         92,  36,  91,   4,  45,  91,  65,  44,  84,  91])


In [89]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [90]:
net = Net(vocab_size, hidden_size, 3)

In [91]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [92]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([479, 80, 115])


tensor([-0.1293, -0.0630,  0.1270, -0.0331,  0.0952,  0.0261,  0.0815, -0.0293,
         0.0246, -0.0213,  0.0041,  0.1070,  0.0560, -0.0501, -0.0589,  0.1305,
         0.0564, -0.0952, -0.1295, -0.0054, -0.0462, -0.0778,  0.0426,  0.0653,
        -0.0028, -0.0244, -0.0426,  0.0826,  0.0970, -0.0085,  0.0146,  0.0193,
         0.1522,  0.0846, -0.0127,  0.0018, -0.0124, -0.1040,  0.1210, -0.0897,
         0.0004, -0.0609,  0.0576, -0.0637,  0.0450,  0.0293, -0.0161, -0.1144,
        -0.0077,  0.0241,  0.1295, -0.0604, -0.1675, -0.0196, -0.0192,  0.0916,
        -0.0073, -0.0542,  0.0146, -0.0761, -0.1230,  0.0177, -0.0457, -0.1993,
         0.0712, -0.0011,  0.0136, -0.1863, -0.0796, -0.0342,  0.0102, -0.0149,
        -0.0183,  0.0198,  0.0656, -0.0474,  0.0108, -0.0450, -0.0282,  0.1353,
         0.1169,  0.0738,  0.0488, -0.0718,  0.1015,  0.1789,  0.1003,  0.0344,
        -0.0914, -0.0248,  0.2098,  0.1220, -0.0175, -0.0904,  0.1046,  0.0973,
         0.0688, -0.0612,  0.0716, -0.00

In [93]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

한한묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻한묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻얄묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻조묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻조묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻한묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻조묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻묻
                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [94]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
얄루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만 올 이도 없 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
얄문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하도이까.
얄리얄리 얄라셩 얄라리 얄라.


In [95]:
print(sentence)

살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

살겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


수정 하였으나 별 차이가 없었다. 이번엔 Net 구조를 변경해보았다.

In [96]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [97]:
net = Net(vocab_size, hidden_size, 5)#hidden layer 수 를 더 크게 변경함

In [98]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [99]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([479, 80, 115])


tensor([ 0.1121, -0.0258, -0.0384, -0.1132, -0.1046, -0.0117, -0.1530, -0.0614,
         0.0842, -0.0812, -0.1264, -0.0393, -0.1039, -0.0571,  0.0838, -0.0294,
         0.0074, -0.0466, -0.0702,  0.0373, -0.1299, -0.0041, -0.0196, -0.1500,
         0.0468, -0.0210, -0.0718, -0.1202, -0.0654, -0.0169,  0.0353, -0.0329,
         0.0280,  0.1212,  0.0337, -0.0293, -0.0266, -0.1459, -0.0151,  0.0332,
         0.0372, -0.1462, -0.0997, -0.0355, -0.0691, -0.1151,  0.0537,  0.0886,
        -0.0240,  0.0508, -0.0649, -0.0095, -0.0309,  0.0052,  0.0130, -0.0019,
         0.0613,  0.1115, -0.0110, -0.0366,  0.0920,  0.0225, -0.0016, -0.0422,
         0.0218,  0.1081, -0.0928, -0.0552,  0.0495,  0.1585, -0.0854, -0.0275,
         0.0362,  0.0687, -0.0050, -0.0137,  0.1180, -0.0705, -0.0411,  0.0265,
         0.0539,  0.1043,  0.0234, -0.1457, -0.0060, -0.0223, -0.0446,  0.0170,
        -0.0972, -0.0628, -0.0464,  0.0505, -0.1154,  0.0361, -0.0093, -0.0081,
         0.0332, -0.0311, -0.0473, -0.01

In [100]:
##15. Training 시작
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

의를를를를의의의의를의를를의의를를의의를의를의의의를의의를의를의의를의를를를의의를의를의의의의의의의를의의의의를의의의를를의의를의를의를를의를의를를의의의를의의를의의를의의를의를를를의를의를의의를의의를의의를의의를의의를의의를의의를의의의의의의를의를의의의를의의의를를의의의의를의를의의를를의의의를를의를를의를를의를를의의를를의의의를를의의를의를를의를를를를를를의를를의를를의의를의의의의를의를를를의의의를의의의를를를의의를의의의를의의의를를의를의의를를를를를의를를를를의를를를를의의의의를의의의를를의의를의의를의의의의의의를를를의의의를의의의를를의의의의의를의를의를를의의의의를의를를의를의를를의를의의를를를의의의를를의를의를를를의의의를를의의의의를를의의의의의를의를의의의를의의의를를의의의를의의의의를의를를의를의를의의를의를의의의를의의의를의의를의를의를를의를의를의의를의를의의의의의의를의를의의의를의의의를를의의의를의의의를의의를의를의를를의의를의를의의의의를의를의를의의의의를를를의를를를의를를의를를의를를의의를의를를의의의의의를를를의의의를의의의를를의의의를의를의의를를를의의의를의의를의를를를를의의를의의의의의의를를의의를를를의의를의의를의를의의를의를를의를의의의를를의의를의의의의의의를를를의의의를의
                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [101]:
print(predict_str)

 라라.

라         라라.

      라. 
얄      라.

라    라.

라   라.

얄    라. 라.
      라.
    라. 라

    라.

라    라. 
     라.

     라.

라   라.

얄     라.
얄라    라라.

라    라.

라    라. 
     라라 

라    라.

라    라.

라   라.

얄     라. 

     라.

     라. 라
얄    라.

라    라. 
얄     라.

라   라.

얄    라라  
얄    라.

라   라.

    라.  
라   라.

얄    라.

     라.

라   라.

얄    라.라라.
얄라   라라라.

라   라.
    라.

라    라라.

     라.

라   라.

얄     라.  
     라.

라   라라.
얄     라.
얄     라. 
    라. 
얄     라.

라   라.

얄     라.
얄라   라. 
     라. 
얄라   라.  
라    라.  
    라.  
     라.
얄라   라.

라   라.


200회 추가 진행 해 보았다.

In [102]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
얄리얄리 얄라. 얄라리 얄라.

가리 새  라   노라.

라라  가다   노라.
얄리  우 가고를 가지    아. 가라   노라.
얄리얄리 얄라. 얄라리 얄라.

가리 새 얄라 낮일랑 리  얄라  
리얄리  라이고 새는       얄라   얄라가.
얄리얄리 얄라. 얄라리 얄라.

가리  살   얄라  
구라  
얄가   가.
얄리  우겠 사   이도 라나 
라  우노라.
얄리얄리 얄라. 얄라리 얄라.

가리우도 얄라노라.

라  얄라노라.
얄리이도    어지도 가켜 얄라  얄겠노라.
얄리얄리 얄라. 얄라리 얄라.

가리       노라.
얄라아   가라   노라.
얄리얄  노  
라  우본노나 를 가을  노라.
얄리얄리 얄라. 얄라리 얄라.

가리 새 노나룩  어지  라  일라   노나. 일가얄리 얄라  누룩을 우도 가고낮일  얄도 나  일라  우 노.
얄리얄리 얄라. 얄라리 얄라.
라노라.
구노 .
얄라  얄겠노라.
얄리  얄리  가  가라리 얄겠노라.
얄리얄리 얄라. 얄라리 얄라.

가 이  살겠노나 새야.
얄이  가  우 노  새야.
  우    얄라 우도 가고 일가  우노라.
얄리얄리 얄라. 얄라리 얄라.

가  새      본라.
얄라도. 가라   본라.
얄리  우   를 가지    아. 가라   본다.
얄리얄리 얄라. 얄라리 얄라.

가  럭 얄라  야랑 리   가  
리얄리   가고 새는       얄라   얄겠 .
얄리얄리 얄라. 얄라리 얄라.

가       얄라  
구라  
얄가   가.
얄리  우겠.사 가가이도  이 
라  우노라.
얄리얄리 얄라. 얄라리 얄라.

가 우도 살 노라.
얄라  얄겠노라.
얄리이도    가지도    얄라리 살겠노라.
얄리얄리 얄라. 얄라리 얄라.

가       듣노라.
얄라라   가다   노라.
얄리얄     
라  해본   를 가을  노라.
얄리얄리 얄라. 얄라리 얄라.

가  새 노 사  술리  겠. 일라   노   얄가얄리 얄라  누룩이 우

In [103]:
print(predict_str)

겠노라.
구를라. 바다에 살겠노라.
얄리얄 다래  먹고 청산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가럭다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
얄리재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라.
외딴 부엌 가다  듣노라.
사리이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모라의 누룩이 매워 (나를) 붙잡으니 내 어찌 할보 까.
얄리얄리 얄라셩 얄라리 얄라.


200회 추가

In [104]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하보 까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.
구를라.
바다에 살겠노라.
얄리얄 다래  먹고 청산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가디저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나리재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라.
외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하보 까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.
구를라.
바다에 살겠노라.
얄리얄 다래  먹고 청산에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가는구나 우는구나 새야.
자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
얄끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가디저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라

hidden state를 추가했음에도 별다른 성능 향상은 얻을 수 없었다.

항상 '가산'이나 '어디다'가 아니라 '가디다' '머루' 대신 '나루' 등 특정한 부분에서만 학습이 저해되는데, 이 부분은 무엇 때문인지 잘은 모르겠으나 RNN이 단순해서 학습이 안되기보다는 하이퍼파라미터 조절로 문장을 끊는 단위를 세세하게 결정하면 해결할 수 있지 않을까 생각했다.

하이퍼 파라미터를 수정해서 다시 해보았다.

In [105]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size 
sequence_length = 40  
learning_rate = 0.005

In [106]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라 -> 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
1 겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라. -> 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.

2 노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
 -> 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄
3 라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄 ->  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리
4  살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리 -> 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄
5 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄 -> 겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리
6 겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 -> 노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 
7 노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리  -> 라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄
8 라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄 -> . 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑
9 . 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑 ->  청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩
10  청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 -> 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 
11 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩

In [107]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[54, 56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1]
[56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70, 76, 110, 113, 91, 32, 33, 102, 91, 46, 45, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36]


In [108]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [109]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([519, 40, 115])
레이블의 크기 : torch.Size([519, 40])


In [110]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])


In [111]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 56,  42,   1,  91,  54,  56,  42,   1,  36,  91,  58,  21,  18,  91,
         54,  56,  42,   1,  36,  70,  76, 110, 113,  91,  32,  33, 102,  91,
         46,  45,  91,  58,  21,  18,  91,  54,  56,  42,   1,  36])


In [112]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [113]:
net = Net(vocab_size, hidden_size, 3)

In [114]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [115]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([519, 40, 115])


tensor([-0.1272,  0.0800, -0.0112, -0.0391, -0.0517,  0.0832, -0.0495, -0.1050,
         0.1435, -0.0134,  0.0428,  0.0185,  0.0218,  0.0654,  0.0414,  0.0118,
        -0.0073,  0.0839,  0.0330, -0.0359, -0.0764, -0.0014,  0.0772,  0.0172,
        -0.0878,  0.0728, -0.0671,  0.0009,  0.1667,  0.0413,  0.0676,  0.0730,
        -0.1300, -0.0602,  0.1721, -0.1134, -0.1384, -0.0128, -0.0384, -0.0792,
        -0.0593, -0.0220, -0.0149,  0.0828, -0.0479,  0.0834, -0.0546,  0.0558,
        -0.0601, -0.0398,  0.0314,  0.0673,  0.0835, -0.0387, -0.0109,  0.0607,
        -0.0151, -0.0295, -0.0434,  0.0863,  0.0671,  0.0914,  0.0162, -0.0409,
         0.0677, -0.1485, -0.0663, -0.0357,  0.1315,  0.1100,  0.1768,  0.0356,
         0.0384, -0.1101, -0.1498, -0.0660, -0.0110, -0.0510, -0.0122, -0.0434,
         0.0496, -0.0328,  0.0441,  0.0922, -0.1297, -0.0341, -0.0615,  0.1343,
         0.0081, -0.0956, -0.1051,  0.0757, -0.0607,  0.1187,  0.0801, -0.0547,
        -0.0969, -0.0127, -0.1389,  0.01

In [116]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
외











금외

금







외






금외

왔






외
외외




켜




외
외외
















외




외
금외






외






금외


















외

외
외외외





외

금







켜



외










외







외






                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
                                                                                                                                                                                                                         

In [117]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


epoch 100 추가

In [118]:
##15. Training 시작
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에

In [119]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


sequence_length 20으로 수정

In [120]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size 
sequence_length = 20  
learning_rate = 0.005

In [121]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 살겠노라 살겠노라. 청산에 살겠노라. -> 겠노라 살겠노라. 청산에 살겠노라.

1 겠노라 살겠노라. 청산에 살겠노라.
 -> 노라 살겠노라. 청산에 살겠노라.
머
2 노라 살겠노라. 청산에 살겠노라.
머 -> 라 살겠노라. 청산에 살겠노라.
머루
3 라 살겠노라. 청산에 살겠노라.
머루 ->  살겠노라. 청산에 살겠노라.
머루랑
4  살겠노라. 청산에 살겠노라.
머루랑 -> 살겠노라. 청산에 살겠노라.
머루랑 
5 살겠노라. 청산에 살겠노라.
머루랑  -> 겠노라. 청산에 살겠노라.
머루랑 다
6 겠노라. 청산에 살겠노라.
머루랑 다 -> 노라. 청산에 살겠노라.
머루랑 다래
7 노라. 청산에 살겠노라.
머루랑 다래 -> 라. 청산에 살겠노라.
머루랑 다래를
8 라. 청산에 살겠노라.
머루랑 다래를 -> . 청산에 살겠노라.
머루랑 다래를 
9 . 청산에 살겠노라.
머루랑 다래를  ->  청산에 살겠노라.
머루랑 다래를 먹
10  청산에 살겠노라.
머루랑 다래를 먹 -> 청산에 살겠노라.
머루랑 다래를 먹고
11 청산에 살겠노라.
머루랑 다래를 먹고 -> 산에 살겠노라.
머루랑 다래를 먹고 
12 산에 살겠노라.
머루랑 다래를 먹고  -> 에 살겠노라.
머루랑 다래를 먹고 청
13 에 살겠노라.
머루랑 다래를 먹고 청 ->  살겠노라.
머루랑 다래를 먹고 청산
14  살겠노라.
머루랑 다래를 먹고 청산 -> 살겠노라.
머루랑 다래를 먹고 청산에
15 살겠노라.
머루랑 다래를 먹고 청산에 -> 겠노라.
머루랑 다래를 먹고 청산에 
16 겠노라.
머루랑 다래를 먹고 청산에  -> 노라.
머루랑 다래를 먹고 청산에 살
17 노라.
머루랑 다래를 먹고 청산에 살 -> 라.
머루랑 다래를 먹고 청산에 살겠
18 라.
머루랑 다래를 먹고 청산에 살겠 -> .
머루랑 다래를 먹고 청산에 살겠노
19 .
머루랑 다래를 먹고 청산에 살겠노 -> 
머루랑 다래를 먹고 청산에 살겠노라
20 
머루랑 다래를 먹고 청산에 살겠노라 -> 머루랑 다래를 먹고 청산에 살겠노라.
21

In [122]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[54, 56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36]
[56, 42, 1, 91, 54, 56, 42, 1, 36, 91, 58, 21, 18, 91, 54, 56, 42, 1, 36, 70]


In [123]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [[np.eye(1,vocab_size,k).flatten() for k in x_data[j]] for j in range(len(x_data))]
#flatten()함수 사용으로 차원을 한꺼풀 벗겨냈다. 188 10 1 26 -> 188 10 26
  

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [124]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([539, 20, 115])
레이블의 크기 : torch.Size([539, 20])


In [125]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [126]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([56, 42,  1, 91, 54, 56, 42,  1, 36, 91, 58, 21, 18, 91, 54, 56, 42,  1,
        36, 70])


In [127]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

"""
torch.nn.RNN(input_size, hidden_layer, num_layer, 
bias=True, batch_first=False, dropout = 0, bidirectional = False)

input_size: In input x the number of expected features.
hidden_layer: The number of features in the hidden state.
num_layer: The num_layer is used as several recurrent layers.
bias: If the bias is False then the layer does not use bias weights.
batch_first: If batch_first is True then input and output tensors are provided ( batch, seq, feature) instead of (seq, batch, feature). The default value of batch_first is False.
dropout: If non-zero, initiate the dropout layer on the output of each RNN layer excluding the last layer with a dropout probability equal to dropout. The default value of dropout is 0.
bidirectional: If True, then it becomes a bidirectional RNN. The default value of bidirectional is False.
"""


class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim,layers, batch_first=True,)
    self.fc = torch.nn.Linear(hidden_dim,input_dim)
    self.input_dim = input_dim
  
  def forward(self, x):
    x = x.view(-1, sequence_length, self.input_dim)
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [128]:
net = Net(vocab_size, hidden_size, 3)

In [129]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [130]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)
outputs[0][0]

torch.Size([539, 20, 115])


tensor([ 0.0060,  0.1099,  0.0781, -0.0179,  0.0626,  0.0877,  0.0512,  0.0674,
         0.0466,  0.0786,  0.0388, -0.0275,  0.0703,  0.0995, -0.0101, -0.0258,
         0.0878,  0.0926,  0.0680,  0.0637, -0.0184, -0.0681,  0.0027,  0.0006,
         0.0623, -0.0185, -0.0681,  0.1424,  0.1266,  0.0388, -0.0211,  0.0241,
        -0.0442, -0.0348,  0.0038,  0.0967, -0.0110, -0.0205, -0.0298,  0.0854,
        -0.0163, -0.0857, -0.0621,  0.0227, -0.1058, -0.1049,  0.0485, -0.0546,
         0.0912,  0.0641,  0.0036, -0.1440,  0.0225, -0.0858, -0.0111,  0.0756,
         0.1573, -0.0801,  0.1233,  0.0682,  0.0624, -0.0144,  0.0166,  0.0364,
        -0.0313, -0.1249, -0.0630, -0.1077,  0.0420,  0.0980,  0.1125, -0.1047,
         0.0161,  0.0802, -0.1457, -0.0228,  0.0624, -0.0718, -0.0835, -0.0189,
        -0.0914,  0.1664,  0.0878, -0.0649, -0.0335,  0.0731,  0.0690, -0.1022,
         0.0063, -0.0750,  0.0629,  0.1147,  0.0335, -0.1435,  0.1224,  0.1634,
        -0.0025, -0.0539,  0.0217,  0.02

In [131]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
얄는에 살겠노라.
얄노랑 살는를 얄고 얄는에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

얄노구나 얄는구나 얄라.
얄고 얄는에 얄노구나 얄라.
얄노구 
름 새한 얄도 새고 얄는에 얄노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리 새 본다 새 가다.
얄 아도 가던 새 본다.
얄리 새고 새기를 
일한 새 아찌 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

얄리 는 얄랑 살일랑 
는 얄랑.
이 이도 새 이도 새랑 가일랑 가 어찌 이 이인가.
얄리얄리 얄라셩 얄라리 얄라.

얄는를 
얄는 새인가 
는에 살는노는 새인가 
얄노이 이도 얄랑에 새도 새고 
일한 얄노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리노라.
겠노라.
얄노에 살겠노라.
얄리아, 새고 얄는를 새고 얄겠에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리  얄다리 얄노라.
얄는 얄는 가다셩 얄노라.
얄리이 얄는에 얄라리 얄는을 얄는 새고 얄노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리  니 얄대랑 살는에 새한 얄고 얄는구. 
얄노박꽃 얄겠의 누겠의 살대 가는를) 어도가니 이 어한 이 가리.
얄리얄리 얄라셩 얄라리 얄라.
 노라.
겠노라.
얄는에 살겠노라.
얄노랑 살겠를 새고 얄는에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

얄노구나 얄라구나 얄라.
얄고 얄라에 얄노라나 얄라.
얄노리 
름 새, 이도 새고 얄는에 얄노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리 새 가다 새 가다.
얄 이래 가던 새 가다.
얄리 새고 새기를 
지고 새 본래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

얄리리럭 얄도 살일랑 
는 이고.
이 이도 새 이도 얄랑 얄일랑 가 어찌 이 이인가.
얄리얄리 얄라셩 얄라리 얄라.

얄는를 
얄는 새인가 
는에 살룩노라 새인가.
얄노얄 이도 얄랑셩 이도 새고 가일에 얄노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리노라.
겠노라.
얄노리 살겠노라.
얄리얄도 새고 얄는를 물고 얄겠에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

얄리  얄다가 얄

In [132]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


In [133]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도

In [134]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


In [ ]:
##15. Training 시작
for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.
겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도

In [ ]:
print(predict_str)

겠노라.살겠노라. 바산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

가럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

가디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다  가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


In [135]:
print(sentence)

살겠노라 살겠노라. 청산에 살겠노라.
머루랑 다래를 먹고 청산에 살겠노라.
얄리얄리 얄랑셩 얄라리 얄라.

우는구나 우는구나 새야. 자고 일어나 우는구나 새야.
너보다 시름 많은 나도 자고 일어나 우노라.
얄리얄리 얄라셩 얄라리 얄라.

가던 새 가던 새 본다. 물 아래 가던 새 본다.
이끼 묻은 쟁기를 가지고 물 아래 가던 새 본다.
얄리얄리 얄라셩 얄라리 얄라.

이럭저럭 하여 낮일랑 지내 왔건만
올 이도 갈 이도 없는 밤일랑 또 어찌 할 것인가.
얄리얄리 얄라셩 얄라리 얄라.

어디다 던지는 돌인가 누구를 맞히려는 돌인가.
미워할 이도 사랑할 이도 없이 맞아서 우노라.
얄리얄리 얄라셩 얄라리 얄라.

살겠노라 살겠노라. 바다에 살겠노라.
나문재, 굴, 조개를 먹고 바다에 살겠노라.
얄리얄리 얄라셩 얄라리 얄라.

가다가 가다가 듣노라. 외딴 부엌 가다가 듣노라.
사슴이 장대에 올라서 해금을 켜는 것을 듣노라.
얄리얄리 얄라셩 얄라리 얄라.

가다 보니 불룩한 술독에 독한 술을 빚는구나.
조롱박꽃 모양의 누룩이 매워 (나를) 붙잡으니 내 어찌 하리이까.
얄리얄리 얄라셩 얄라리 얄라.


Epoch hyperparameter RNN 변경을 모두 진행했으나, 도저히 첫문단의 '바산'에 살겠노라 부분의 학습이 어렵다. 아무래도 운율을 위한 비슷한 문장 '바다에 살겠노라', '청산에 살겠노라'의 부분이 아예 동일한 특징 때문에 학습에 어려움을 겪는 것 같았다.